In [9]:
from models.resnet import MyResNet
from data_utils import CustomDataset, get_transforms
from tqdm.notebook import tqdm
from torchvision.models import efficientnet_b1
from torchvision.models import efficientnet_b0
import torch
import pandas as pd
import kaggle

device = 'cuda:1'

In [10]:
'checkpoints/worthy-yogurt-79-20-0.8076_model.pt.pt'

'checkpoints/worthy-yogurt-79-19-0.8071_model.pt'

In [11]:
checkpoint = 'worthy-yogurt-79-19-0.8071'

In [12]:
def inference(model, dataloader, device='cpu'):
    preds = []
    model.eval()
    for x_batch, _ in tqdm(dataloader):
        data = x_batch.to(device)
        with torch.inference_mode():
            output = model(data)
            preds.append(output.detach().cpu())
            
    return torch.cat(preds)

def predict(model_cls, checkpoint, batchsize=128, base_sumbission='data/sample_submission.csv'):
    model = efficientnet_b0(num_classes=5)
    model.load_state_dict(torch.load(checkpoint))
    model.to(device)
    dataloader = torch.utils.data.DataLoader(CustomDataset(base_sumbission, get_transforms('val')), batch_size=batchsize, num_workers=4)
    preds = inference(model, dataloader, device)
    return preds

preds = predict(MyResNet, f'checkpoints/{checkpoint}_model.pt')

  0%|          | 0/151 [00:00<?, ?it/s]

In [13]:
test = pd.read_csv('data/sample_submission.csv', )
test.iloc[:, 1:] = preds
test.to_csv(f'data/submissions/{checkpoint}.csv', index=False)

In [15]:
kaggle.api.competition_submit(f'data/submissions/{checkpoint}.csv', 'Effnetb0 320x320 augmented cyclelr', 'chest-x-ray-image-classification')

100%|██████████| 2.10M/2.10M [00:01<00:00, 1.11MB/s]


Successfully submitted to Chest x-ray image classification

# Averaging

In [16]:
def averaging(base_model, paths):
    target_state_dict = base_model(num_classes=5).state_dict()
    for key in target_state_dict:
        target_state_dict[key].data.fill_(0.)
        
    for path in tqdm(paths):
        model = base_model(num_classes=5)
        model.load_state_dict(torch.load(path))
        state_dict = model.state_dict()
        for key in target_state_dict:
            if target_state_dict[key].data.dtype != torch.float32:
                continue
            target_state_dict[key].data += state_dict[key].data.clone() / len(paths)
            
    return target_state_dict

In [17]:
paths = [
    'checkpoints/worthy-yogurt-79-15-0.8039_model.pt',
    'checkpoints/worthy-yogurt-79-16-0.8044_model.pt',
    'checkpoints/worthy-yogurt-79-17-0.8060_model.pt',
    'checkpoints/worthy-yogurt-79-18-0.8066_model.pt',
    'checkpoints/worthy-yogurt-79-19-0.8071_model.pt',
    'checkpoints/worthy-yogurt-79-20-0.8076_model.pt',
]

cool_model = averaging(efficientnet_b0, paths)

  0%|          | 0/6 [00:00<?, ?it/s]

In [36]:
model = efficientnet_b0(num_classes=5)
model.load_state_dict(cool_model)
model.to(device)
dataloader = torch.utils.data.DataLoader(CustomDataset('data/sample_submission.csv', get_transforms('train')), batch_size=256, num_workers=8)

tta = 30
for t in range(tta):
    preds += inference(model, dataloader, device)

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

...

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

In [37]:
checkpoint += '_30'

In [38]:
test = pd.read_csv('data/sample_submission.csv', )
test.iloc[:, 1:] = preds / (tta + 11)
test.to_csv(f'data/submissions/{checkpoint}.csv', index=False)

In [40]:
kaggle.api.competition_submit(f'data/submissions/{checkpoint}.csv', 'Effnetb0 320x320 augmented averaging tta', 'chest-x-ray-image-classification')

100%|██████████| 2.11M/2.11M [00:02<00:00, 901kB/s]


Successfully submitted to Chest x-ray image classification

# План B!

In [5]:
whatw = torch.tensor([[ 0.8884573 , -0.02050001,  0.06470138,  0.07708601, -0.03539958],
       [ 0.01634081,  0.90970256,  0.04924741, -0.03046782, -0.02745024],
       [ 0.00647937, -0.00445542,  0.83861348,  0.03517183,  0.01810227],
       [ 0.00894726, -0.00640146, -0.01189516,  0.85447432,  0.01298881],
       [-0.0050264 ,  0.00760905,  0.02597707, -0.05035251,  0.93361056]])
whatb = torch.tensor([ 0.13252833, -0.04922415, -0.38749676, -0.05492893,  0.1900582 ])

def predict(model_cls, checkpoint, batchsize=256, base_sumbission='data/sample_submission.csv'):
    model = model_cls(densenet121())
    model.load_state_dict(torch.load(checkpoint))
    model.base_model.postclassifier = torch.nn.Linear(5, 5)
    model.base_model.postclassifier.weight = torch.nn.Parameter(whatw)
    model.base_model.postclassifier.bias = torch.nn.Parameter(whatb)

#     model.base_model.classifier.weight = torch.nn.Parameter(whatw @ model.base_model.classifier.weight)
#     model.base_model.classifier.bias = torch.nn.Parameter(whatb + model.base_model.classifier.bias)
    model.to(device)
    dataloader = torch.utils.data.DataLoader(CustomDataset(base_sumbission, get_transforms('train')), batch_size=batchsize, num_workers=4)
    preds = inference(model, dataloader, device)
    return preds

preds = predict(MyResNet, f'checkpoints/{checkpoint}_model.pt')

  0%|          | 0/76 [00:00<?, ?it/s]

In [6]:
test = pd.read_csv('data/sample_submission.csv', )
test.iloc[:, 1:] = preds
test.to_csv(f'data/submissions/{checkpoint}_hehe.csv', index=False)

In [7]:
kaggle.api.competition_submit(f'data/submissions/{checkpoint}_hehe.csv', 'Densenet121 320x320 augmented + hehe', 'chest-x-ray-image-classification')

100%|██████████| 2.11M/2.11M [00:02<00:00, 977kB/s]


Successfully submitted to Chest x-ray image classification